In [ ]:
import networkx as nx
from networkx.algorithms import approximation
import numpy as np
import scipy as sp
import pickle
import math
import random
import matplotlib.pyplot as plt
import pandas as pd
import tqdm

# Prepare Data

In [ ]:
try:
  with open('erdos_renyi_a-g_with_graph_features.pkl', 'rb') as file:
    qaoa_df = pickle.load(file)

except FileNotFoundError:
  print("File not found")
except Exception as e:
  print(f"An error occurred while opening or loading the file: {e}")

# Graph Feature Functions

In [ ]:
# Graph features
feature_names = {
0:'Number of vertices',
1:'Number of edges',
2:'Edge density',
3:'Mean degree',
4:'Standard deviation of degrees',
5:'Skewness of degrees',
6:'Minimum degree',
7:'Maximum degree',
8:'Diameter',
9:'Radius',
10: 'Vertex connectivity',
11: 'Edge connectivity',
12: 'Global clustering coefficient',
13: 'Mean local clustering coefficient',
14: 'Standard deviation of local clustering coefficients',
15: 'Skewness of local clustering coefficients',
16: 'Minimum local clustering coefficient',
17: 'Maximum local clustering coefficient',
18: 'Treewidth',
19: 'Average path length',
20: 'Circuit rank',
21: 'Girth',
22: 'Mean betweenness centrality',
23: 'Standard deviation of betweenness centralities',
24: 'Skewness of betweenness centralities',
25: 'Minimum betweenness centrality',
26: 'Maximum betweenness centrality',
27: 'Algebraic connectivity',
28: 'Von Neumann entropy',
29: 'Adjacency spectrum mean',
30: 'Adjacency spectrum standard deviation',
31: 'Adjacency spectrum skewness',
32: 'Adjacency spectrum min',
33: 'Adjacency spectrum max',
34: 'Laplacian spectrum mean',
35: 'Laplacian spectrum standard deviation',
36: 'Laplacian spectrum skewness',
37: 'Laplacian spectrum min',
38: 'Laplacian spectrum max',
39: 'Planarity',
40: 'Mean harmonic centrality',
41: 'Standard deviation of harmonic centralities',
42: 'Skewness of harmonic centralities',
43: 'Minimum harmonic centrality',
44: 'Maximum harmonic centrality',
45: 'Harmonic diameter',
46: 'Mean core number',
47: 'Standard deviation of core numbers',
48: 'Skewness of core numbers',
49: 'Minimum core number',
50: 'Maximum core number',
51: 'Chordality',
52: 'Haemers bound',
53: 'Claw-free'
}

def flatten(x):
  if not isinstance(x, list):
    return [x]
  else:
    return [z for y in x for z in flatten(y)]

def skew1(input_list):
  if abs(max(input_list) - min(input_list)) < 0.000000001:
    return 0
  return sp.stats.skew(input_list)

def statistics(input_list, **kwargs):
  output = []
  if kwargs.get('include_mean', True):
    output.append(np.mean(input_list))
  if kwargs.get('include_std', True):
    output.append(np.std(input_list))
  if kwargs.get('include_skew', True):
    output.append(skew1(input_list))
  if kwargs.get('include_min', True):
    output.append(min(input_list))
  if kwargs.get('include_max', True):
    output.append(max(input_list))
  return output

In [ ]:
# Note transitivity is also known as the global clustering coefficient

def num_vertices(G):
  """
  Input: G (networkx graph)
  Output: number of vertices
  """
  return G.number_of_nodes()

def num_edges(G):
  """
  Input: G (networkx graph)
  Output: number of edges
  """
  return G.number_of_edges()

def degree_statistics(G):
  """
  Input: G (networkx graph)
  Output: list of mean, std, skew, min, & max of distribution of degrees
  """
  degrees = []
  for pair in G.degree():
    degrees.append(pair[1])
  return statistics(degrees)

def diameter(G):
  """
  Input: G (networkx graph)
  Output: sum of diameters among connected components of G
  """
  diameter_sum = 0
  for C in (G.subgraph(c).copy() for c in nx.connected_components(G)):
    diameter_sum += nx.diameter(C)
  return diameter_sum

# Note the radius is a lower bound on the independence number
def radius(G):
  """
  Input: G (networkx graph)
  Output: sum of radii among connected components of G
  """
  radius_sum = 0
  for C in (G.subgraph(c).copy() for c in nx.connected_components(G)):
    radius_sum += nx.radius(C)
  return radius_sum

# Uses heuristic algorithm to approximate treewidth. Might want to find an exact algorithm later.
def treewidth_approx(G):
  """
  Uses heuristic algorithm to approximate treewidth

  Input: G (networkx graph)
  Output: approximation of treewidth of G
  """
  return approximation.treewidth_min_fill_in(G)[0]

def clustering_statistics(G):
  """
  Input: G (networkx graph)
  Output: list of mean, std, skew, min, & max of distribution of clustering coefficients
  """
  clustering_coeffs = list(nx.clustering(G).values())
  return statistics(clustering_coeffs)

def average_shortest_path_length(G):
  """
  Input: G (networkx graph)
  Output: average shortest path length of G
  """
  if G.number_of_nodes() < 2:
    return 0
  avg = 0
  for C in (G.subgraph(c).copy() for c in nx.connected_components(G)):
    avg += nx.average_shortest_path_length(C) * C.number_of_nodes() * (C.number_of_nodes() - 1)
  avg = avg / (G.number_of_nodes() * (G.number_of_nodes() - 1))
  return avg

def circuit_rank(G):
  """
  Input: G (networkx graph)
  Output: circuit rank of G
  """
  return G.number_of_edges() - G.number_of_nodes() + nx.number_connected_components(G)

def girth(G):
  """
  Input: G (networkx graph)
  Output: girth of G
  """
  if nx.girth(G) != math.inf:
    return nx.girth(G)
  else:
    return 0

def betweenness_centrality_statistics(G):
  """
  Input: G (networkx graph)
  Output: list of mean, std, skew, min, & max of distribution of betweenness centralities
  """
  bc = nx.betweenness_centrality(G)
  bc_values = list(bc.values())
  return statistics(bc_values)

def algebraic_connectivity(G):
  """
  The algebraic connectivity of a connected undirected graph is the second
  smallest eigenvalue of its Laplacian matrix.

  Input: G (networkx graph)
  Output: algebraic connectivity of G
  """
  if G.number_of_nodes() < 2:
    return 0
  return nx.algebraic_connectivity(G, method='tracemin_lu')

def von_neumann_entropy(G):
  """
  Input: G (networkx graph)
  Output: von neumann entropy of G
  """
  degree_sum = 0
  for pair in G.degree():
    degree_sum += pair[1]
  if degree_sum == 0:
    return 0
  rho = (1 / degree_sum) * nx.laplacian_matrix(G) # density matrix
  rho = rho.todense()
  evals = np.linalg.eigvals(rho)
  entropy = 0
  for eval in evals:
    real_eval = np.real(eval)
    if real_eval > 0:
      entropy -= real_eval * math.log2(real_eval)
  return entropy

def adjacency_spectrum_statistics(G):
  """
  Input: G (networkx graph)
  Output: list of mean, std, skew, min, & max of distribution of adjacency spectrum
  """
  evals = nx.adjacency_spectrum(G)
  evals = [np.real(num) for num in evals]
  return statistics(evals)

def laplacian_spectrum_statistics(G):
  """
  Input: G (networkx graph)
  Output: list of mean, std, skew, min, & max of distribution of laplacian spectrum
  """
  evals = nx.laplacian_spectrum(G)
  return statistics(evals)

def planarity(G):
  """
  Input: G (networkx graph)
  Output: 1 if planar, 0 if not
  """
  return int(nx.is_planar(G))

def harmonic_centrality_statistics(G):
  """
  Input: G (networkx graph)
  Output: list of mean, std, skew, min, & max of distribution of harmonic centralities
  """
  return statistics(list(nx.harmonic_centrality(G).values()))

def harmonic_diameter(G):
  """
  Input: G (networkx graph)
  Output: harmonic diameter of G
  """
  result = nx.harmonic_diameter(G)
  if result == math.inf or G.number_of_nodes() < 2:
    return 0
  else:
    return result

def core_number_statistics(G):
  """
  Input: G (networkx graph)
  Output: list of mean, std, skew, min, & max of distribution of core numbers
  """
  return statistics(list(nx.core_number(G).values()))

def chordality(G):
  """
  Input: G (networkx graph)
  Output: 1 if chordal, 0 if not
  """
  return int(nx.is_chordal(G))

def haemers_bound(G):
  '''
  Computes an upper bound for the independence number of G. This bound was
  obtained by Haemers as a generalization of the Hoffman ratio bound given. In
  the case of division by zero, returns the number of vertices of G.

  Input: G (networkx graph)
  Output: upper bound for the independence number of G
  '''
  n = G.number_of_nodes()
  degrees = []
  for pair in G.degree():
    degrees.append(pair[1])
  min_degree = min(degrees)
  evals = nx.adjacency_spectrum(G)
  evals = [np.real(num) for num in evals]
  min_eval = min(evals)
  max_eval = max(evals)

  denom = min_degree ** 2 - (max_eval * min_eval)
  if abs(denom) < 0.000000000001:
    return n
  return (-n * max_eval * min_eval) / denom

def claw_free(G):
  '''
  Input: G (networkx graph)
  Output: 1 if G is claw-free, 0 if not
  '''
  H = nx.complete_bipartite_graph(1,3)
  isomatcher = nx.isomorphism.GraphMatcher(G, H)
  return int(not isomatcher.subgraph_is_isomorphic())


graph_features_unflattened = [num_vertices, num_edges, nx.density, degree_statistics, diameter,
                  radius, nx.node_connectivity, nx.edge_connectivity, nx.transitivity,
                  clustering_statistics, treewidth_approx, average_shortest_path_length,
                  circuit_rank, girth, betweenness_centrality_statistics, algebraic_connectivity,
                  von_neumann_entropy, adjacency_spectrum_statistics, laplacian_spectrum_statistics,
                              planarity, harmonic_centrality_statistics, harmonic_diameter,
                              core_number_statistics, chordality, haemers_bound, claw_free]

def graph_features(G):
  """
  Calls above functions & returns flattened list of features to
  append to dataframe

  Input: G (networkx graph)
  Output: list of graph features for G
  """
  return flatten([f(G) for f in graph_features_unflattened])


# Computing Graph Features

In [ ]:
qaoa_graph_data = np.array(qaoa_df['adj'])
features = []

for i in range(len(qaoa_df)):
  features.append(graph_features(nx.Graph(qaoa_graph_data[i])))

features_df = pd.DataFrame(data=features)
features_df = features_df.rename(columns=feature_names)

concatenated_df = pd.concat([qaoa_df, features_df], axis=1)
concatenated_df.to_csv("/content/df_with_graph_features.csv")
concatenated_df.to_pickle("/content/df_with_graph_features.pkl")